
# Введение в обработку естественного языка
## Создание чат-бота в Telegram

Подключенная к телеграмм языковая модель на основе Bert из deeppavlov, обученная на сказках. Умеет переводить простые вопросы и отвечать на некоторые вопросы из сказок.

In [4]:
!pip install --q deeppavlov

In [5]:
!python -m deeppavlov install squad_bert

Ignoring transformers: markers 'python_version < "3.8"' don't match your environment


In [6]:
from deeppavlov import configs, build_model

model = build_model(configs.squad.squad_bert, download=True)

2023-12-12 20:10:36.47 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/v1/squad/squad_torch_bert_cased.tar.gz download because of matching hashes
INFO:deeppavlov.download:Skipped http://files.deeppavlov.ai/v1/squad/squad_torch_bert_cased.tar.gz download because of matching hashes
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForQuestionAnswering: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionA

In [7]:
text = open('/content/THE FISHERMAN AND THE GOLDFISH - SOVIET CHILDREN\'S BOOK_djvu.txt', 'rb').read().decode(encoding='cp1251')
text = ' '.join(text.split())
print(text)

A.PUSHKIN THE FISHERMAN AND THE GOLDFISH ~<V вЂў > A. PUSHKIN THE FISHERMAN AND THE GOLDFISH MK 'AVMmwl mm wm There once lived an old man and his goodwife On the shore of the deep blue ocean; They lived in a tumble-down hovel For thirty-three summers and winters. The old man used to fish for his living, And his wife spun yarn on her distaff. He once cast his net in the ocean, And pulled it up with mud from the bottom; He again cast his net in the ocean, And this time caught nothing but seaweed; When he cast his net for the third time, One fish was all that he landed, No common fish, though, but a goldfish. Now the goldfish began to implore him, And it spoke like a real human being: вЂњPut me back, old man, into the oceanвЂ” I will pay you a right royal ransom, I will give you whatever you ask me.вЂќ The old man was astonished and frightenedвЂ” HeвЂ™d been fishing for thirty-three summers, But had not heard of any fish talking. So with care he untangled the goldfish And tenderly said as

In [8]:
!pip install -U sacremoses

In [9]:
from transformers import FSMTForConditionalGeneration, FSMTTokenizer

en_ru = "facebook/wmt19-en-ru"
tokenizer_ru = FSMTTokenizer.from_pretrained(en_ru)
to_ru = FSMTForConditionalGeneration.from_pretrained(en_ru)

ru_en = "facebook/wmt19-ru-en"
tokenizer_en = FSMTTokenizer.from_pretrained(ru_en)
to_en = FSMTForConditionalGeneration.from_pretrained(ru_en)

def translation(text, model, tokenizer):
    input_ids = tokenizer.encode(text, return_tensors="pt")
    outputs = model.generate(input_ids)
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return decoded

In [10]:
model(['DeepPavlov is a library for NLP and dialog systems.'], ['What is DeepPavlov?'])

[['a library for NLP and dialog systems'], [14], [1.0]]

In [11]:
model([text], ['Who is author?'])

Token indices sequence length is longer than the specified maximum sequence length for this model (2263 > 512). Running this sequence through the model will result in indexing errors


[['Louis Zelikoff'], [883], [1.0]]

In [12]:
s = 'Кто автор?'
translation(s, to_en, tokenizer_en)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (200) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'Who is the author?'

In [13]:
model([text], [translation(s, to_en, tokenizer_en)])

[['Louis Zelikoff'], [883], [1.0]]

In [14]:
model([text], [translation('Почему разгневалась рыбка?', to_en, tokenizer_en)])

[['Its billows'], [67], [1.0]]

In [15]:
model([text], [translation('Чем закончилась сказка?', to_en, tokenizer_en)])

[['He sadly set out for the seashore'], [954], [0.9999879002571106]]

In [16]:
model([text], [translation('Что было разбито?', to_en, tokenizer_en)])

[['wash-tub'], [831], [1.0]]

In [17]:
o_t = model([text], [translation('Что было разбито?', to_en, tokenizer_en)])
o_t[0][0]

'wash-tub'

In [18]:
translation('seaweed', to_ru, tokenizer_ru)

'seaweed'

In [19]:
translation('food', to_ru, tokenizer_ru)

'продовольствие'

In [20]:
!pip install python-telegram-bot==13.13

In [21]:
import os
from telegram.ext  import Updater, CommandHandler, MessageHandler, Filters, CallbackContext
from telegram import Update

In [3]:
list_for_fairytails = ['быль', 'миф', 'колобок', 'волшебство', 'рассказ', 'вымысел', 'андерсен', 'присказка', 'лес', 'легенда', 'сказание', 'сон', 'баба-яга', 'золотая', 'рыбка', 'небылица', 'фантазия', 'былина', 'золушка', 'красная', 'шапочка', 'чудо', 'добро', 'репка', 'иван', 'выдумка', 'ложь', 'сказ', 'история', 'волк', 'царь', 'салтан', 'морозко', 'кощей', 'конец', 'белоснежка', 'буратино', 'медведь', 'иванушка', 'богатырь', 'притча', 'лиса', 'басня', 'мечта', 'иван-царевич', 'радость', 'теремок', 'бабушка', 'жизнь', 'иван-дурак', 'мудрость', 'царевна', 'чудеса', 'правда', 'рассказчица', 'старик', 'хоттабыч', 'волшебница', 'предание', 'змей', 'горыныч', 'мультфильм', 'кот', 'ученый', 'лягушка', 'василиса', 'емеля', 'дурак', 'фильм', 'арина', 'родионовна', 'алёнушка', 'избушка', 'враньё', 'жар-птица', 'гуси-лебеди', 'рыбка', 'пушкин', 'повесть', 'волшебник', 'перо', 'конёк-горбунок', 'намёк', 'цветик-семицветик', 'ночь', 'сивка-бурка', 'садко', 'аладдин', 'аленький', 'цветочек', 'ковер-самолёт', 'русалка', 'лебеди', 'золотая', 'антилопа', 'чудовище', 'тысяча', 'и', 'одна', 'ночь', 'айболит', 'каменный', 'цветок', 'народ', 'василиса', 'премудрая', 'фольклор', 'лукоморье', 'щука', 'иванушка', 'дурачок', 'любовь', 'курочка', 'ряба', 'царь', 'снежная', 'королева', 'диво', 'рыба-кит', 'яга', 'песня', 'печка', 'руслан', 'и', 'людмила', 'явь', 'принцесса', 'конь-огонь', 'царевна-лягушка', 'дракон', 'серебряное', 'копытце', 'персонаж', 'бажов', 'синдбад', 'финист', 'маугли', 'три', 'желания', 'забава', 'царство', 'шахерезада', 'вечер', 'дремучий', 'лес', 'фэнтези', 'магия', 'заяц', 'изба', 'хороший', 'конец', 'прибаутка', 'лукошко', 'дед', 'море', 'сочинение', 'счастье', 'васнецов', 'марья', 'моревна', 'колыбель', 'вакула', 'леший', 'загадка', 'джинн', 'облака', 'повествование', 'рассказчик', 'три', 'богатыря', 'зима', 'месяц', 'гусли', 'диафильм', 'реальность', 'сказитель', 'рапунцель', 'ершов', 'мальчиш-кибальчиш', 'остров', 'буян', 'девушка', 'сирены', 'праздник', 'доброта', 'фантастика', 'девочка', 'поле', 'печь', 'феникс', 'маленький', 'принц', 'лампа', 'яблонька', 'птица', 'луна', 'русалочка', 'чиполлино', 'роща', 'дедушка', 'солоха', 'ступа', 'букварь', 'кот-баюн', 'космос', 'рыба', 'лепота', 'берёзовая', 'роща', 'царь', 'горы', 'окно', 'жанр', 'малахит', 'небосвод', 'неправда', 'придумка', 'хоромы', 'ауканье', 'небесная', 'твердь', 'тоторо', 'снегурочка', 'гоголь', 'феи', 'софия', 'ротару', 'челн', 'вязь', 'лубок', 'архетип', 'ящерки', 'гайдар', 'царевна', 'морская', 'чёрт', 'сказы', 'свиток', 'тридевятое', 'царство', 'единорог', 'кузнец', 'вакула', 'иван', 'билибин', 'изумруды', 'великан', 'маша', 'машенька', 'песнь', 'гном', 'терем', 'писатель', 'самоцветы', 'олимпийский', 'мишка', 'врубель', 'капитан', 'немо', 'детство', 'книга', 'скатерть-самобранка', 'репа', 'город', 'василиса', 'прекрасная', 'дети', 'двенадцать', 'месяцев', 'тридесятое', 'государство', 'карлсон', 'мораль', 'книжка', 'добрый', 'молодец', 'шапка-невидимка', 'скороходы', 'урок', 'три', 'толстяка', 'братья', 'гримм', 'балда', 'кот', 'сказочник', 'огниво', 'мойдодыр', 'хозяйка', 'медной', 'горы', 'синяя', 'птица', 'изумрудный', 'город', 'змей', 'эльфы', 'мачеха', 'данила-мастер', 'щелкунчик', 'красна', 'девица', 'бог', 'марья', 'тыква', 'бармалейкин', 'мультик', 'баюн', 'золотой', 'петушок', 'оловянный', 'солдатик', 'оле', 'лукойе', 'дюймовочка', 'мёртвая', 'вода', 'колобки', 'невод', 'волшебная', 'палочка', 'малахитовая', 'шкатулка', 'царевна-лебедь', 'принц', 'рай', 'ходжа', 'насреддин', 'клубок', 'алёша', 'попович', 'ночь', 'перед', 'рождеством', 'чудо-юдо', 'золотой', 'гребешок', 'золотой', 'дождь', 'петя', 'и', 'волк', 'кикимора', 'молодильные', 'яблоки', 'хрустальный', 'мост', 'мальчик-с-пальчик', 'петушок', 'пеппи', 'длинный', 'чулок', 'няня', 'эльф', 'русь', 'колдунья', 'медные', 'трубы', 'приключения', 'буратино', 'гусь', 'полюшко-поле', 'дед', 'мороз', 'ребёнок', 'былинка', 'фея', 'синяя', 'борода', 'чтение', 'настя', 'зурбаган', 'раскраска', 'грусть', 'водяной', 'скороход', 'дерево', 'хоббит', 'серый', 'волк', 'сюжет', 'зло', 'питер', 'пэн', 'соловей-разбойник', 'али-баба', 'муха-цокотуха', 'сказочность', 'поучение', 'гномик', 'нибиру', 'помидор', 'библия', 'старуха', 'иллюзия', 'замок', 'сусеки', 'кино', 'окошко', 'борода', 'дереза', 'индия', 'конёк', 'бессмертие', 'воображение', 'дубрава', 'русалки', 'гуси', 'зеркало', 'меч-кладенец', 'кривда', 'ведьма', 'чеширский', 'кот', 'невидаль', 'александр', 'роу', 'странствия', 'лилипут', 'иносказание', 'чудище', 'морское', 'квакушка', 'мышка-норушка', 'чуковский', 'дурачок', 'оконце', 'иван-да-марья', 'богатыри', 'чародей', 'судьба', 'алиса', 'полумесяц', 'девицы', 'медная', 'гора', 'детский', 'мир', 'марья-искусница', 'дудочка', 'страшила', 'лошадки', 'спящая', 'красавица', 'несмеяна', 'жучка', 'белка', 'огневушка-поскакушка', 'медный', 'всадник', 'семь', 'гномов', 'живая', 'вода', 'алиса', 'в', 'стране', 'чудес', 'ряба', 'корыто', 'тайга', 'подснежники', 'рыбак', 'семицветик', 'гарри', 'поттер', 'колодец', 'смазка', 'винни-пух', 'предисловие', 'обыкновенное', 'чудо', 'бабайка', 'китеж-град']
list_for_weather = ['грязь', 'явление', 'радуга', 'порыв', 'ветра', 'осень', 'лужа', 'сводка', 'дом', 'снегопад', 'антициклон', 'гром', 'студень', 'пляж', 'тихий', 'океан', 'заклинатель', 'воздух', 'религия', 'непредсказуемость', 'неизбежность', 'паводки', 'стекло', 'станция', 'цельсий', 'плюс', 'сырость', 'раскаты', 'грома', 'питер', 'пурга', 'похолодание', 'аномалия', 'зонд', 'засада', 'утро', 'ливень', 'равноденствие', 'время', 'года', 'примета', 'стриж', 'направление', 'вологда', 'слонёнок', 'весна', 'фобос', 'флюгер', 'говно', 'облачка', 'отношения', 'клёв', 'суровость', 'хмурость', 'прояснение', 'раскаты', 'метеостанция', 'тучка', 'зима', 'мелодекламация', 'дневник', 'май', 'велосипед', 'крым', 'чёрное', 'море', 'зной', 'яндекс', 'состояние', 'туча', 'небесная', 'канцелярия', 'октябрь', 'реклама', 'мчс', 'воскресенье', 'заморозки', 'ноябрь', 'жизнь', 'ветреность', 'пейзаж', 'климатология', 'ягода', 'видимость', 'обстановка', 'погода','солнечно', 'осадка', 'дождь', 'снег', 'ливень', 'зонт', 'прогноз', 'дождь', 'природа', 'ветер', 'климат', 'холод', 'гидрометцентр', 'настроение', 'ясность', 'тучи', 'осадки', 'море', 'облачность', 'атмосфера', 'непогода', 'переменчивость', 'зонт', 'снег', 'солнце', 'лето', 'ненастье', 'синоптик', 'слякоть', 'синоптики', 'температура', 'циклон', 'туман', 'метеорология', 'пагода', 'жара', 'облако', 'новости', 'небо', 'облака', 'штиль', 'изменчивость', 'условия', 'мороз', 'пасмурность', 'перемена', 'вероятность', 'прогулка', 'благодать', 'сезон', 'улица', 'прогнозы', 'градус', 'метеоролог', 'гроза', 'термометр', 'непрогнозируемость', 'барометр', 'градусник', 'переменная', 'тепло', 'зонтик', 'шторм', 'давление', 'ведро', 'долина', 'умиротворение', 'грусть', 'вёдра', 'интерферон', 'тучность', 'изморось', 'евро', 'туманность', 'метель', 'телетекст', 'поле', 'полёт', 'чайковский', 'потепление', 'днепр', 'морепродукт', 'диктор', 'дворник', 'дерьмо', 'смерч', 'солнечность', 'тайфун', 'мир', 'капель', 'дым', 'день', 'перемены', 'праздник', 'мода']
list_for_translation = ['перевод', 'интерпритировать', 'транслировать', 'преобрзование', 'язык', 'деньги', 'гоблин', 'словарь', 'переводчик', 'транскрипция', 'интерпретация', 'значение', 'конвертация', 'трансляция', 'банк', 'подстрочник', 'часы', 'английский', 'язык', 'транзакция', 'стрелка', 'работа', 'трудность', 'стрелки', 'трудности', 'субтитры', 'иностранец', 'управленец', 'перенос', 'вестерн', 'правило', 'слово', 'интерпретатор', 'гипертекст', 'пиркод', 'счисление', 'толмач', 'почта', 'транслитерация', 'место', 'жительства', 'зачёт', 'ну', 'погоди', 'труд', 'приход', 'трансфер', 'корректура', 'транзит', 'гранж', 'увольнение', 'уточнение', 'языки', 'бухгалтерия', 'иностранный', 'язык', 'толкование', 'юла', 'интернет', 'квартплата', 'покемон', 'текст', 'русификация', 'титры', 'адаптация', 'сложности', 'искусство', 'мера', 'штамп', 'трапеция', 'расход', 'транслятор', 'стрелочник', 'иврит', 'оригинал', 'литерация', 'объяснение', 'транс', 'фильм', 'измерение', 'сублимация', 'речь', 'озвучка', 'синхрон', 'понимание', 'перенаправление', 'синхронист', 'дубляж']

In [22]:
def is_in_list(text, list):
    for word in text:
        if word in list:
            return True
    return False

In [23]:
import re

def is_english(text):
    return bool(re.search(r'[a-zA-Z]', text))

In [26]:
updater = Updater(token='6302652675:AAHGhGzAQGV5u8kFN49g9JdNp_cZQJ4BzfI')
dispatcher = updater.dispatcher

In [27]:
def startCommand(update: Update, context: CallbackContext):
    update.message.reply_text('Привет!\nЯ ботик (простой и очень глупый бот).\nЕсли надо, я могу пообщаться о сказках, только прошу снисхождения, я очень плохо в них разбираюсь, перевести простые предложения с английского на русский и обратно и рассказать о погоде.\nПоехали?')

def stopCommand(update: Update, context: CallbackContext):
    update.message.reply_text('Пока')

In [28]:
def textMessage(update: Update, context: CallbackContext):
    input = update.message.text
    if is_english(input):#update.message.text):
        update.message.reply_text(translation(input, to_ru, tokenizer_ru) + 'is_english' + input) # update.message.text

    elif is_in_list(input, list_for_translation):
        print('ru_en' + input)
        update.message.reply_text(translation(update.message.text, to_en, tokenizer_en) + 'list_from_translation' + input)

    elif is_in_list(input, list_for_fairytails):
        update.message.reply_text('Прошу учесть мои скромные возможности и выбирать наиболее известные сказки и понимать, что даже самые известные сказки я знаю не все. Какую сказку хотите обсудить?')
        input = update.message.text
        for word in input.split():
          file_name, hope = get_tail(word)
          if hope:
            text = open(file_name, 'rb').read().decode(encoding='cp1251')
            text = ' '.join(text.split())
            break
        if file_name == 'Облом':
            update.message.reply_text(file_name + ', я не знаю такой сказки...')
        else:
          input = update.message.text
          update.message.reply_text('Отлично! Можно попробовать обсудить эту сказку. Готов лицезреть вопрос.' + input)
        output = model([text], [translation(input, to_en, tokenizer_en)])
        update.message.reply_text(translation(output[0][0], to_ru, tokenizer_ru))
    else:
      update.message.reply_text('Что-то я совсем глуп. Можно отдохнуть?' + input)

    return

In [29]:
def get_tail(word):
  for root, dirs, files in os.walk('/content'):
    for names in files:
        if word.lower() in names.lower():
             return os.path.join(root, names), True
  return 'Облом', False

In [30]:
dispatcher.add_handler(CommandHandler("start", startCommand))
dispatcher.add_handler(CommandHandler("stop", stopCommand))
dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, textMessage))

updater.start_polling()
updater.idle()

ERROR:telegram.ext.dispatcher:No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/telegram/ext/dispatcher.py", line 557, in process_update
    handler.handle_update(update, self, check, context)
  File "/usr/local/lib/python3.10/dist-packages/telegram/ext/handler.py", line 199, in handle_update
    return self.callback(update, context)
  File "<ipython-input-28-1f4718c6c075>", line 24, in textMessage
    output = model([text], [translation(input, to_en, tokenizer_en)])
UnboundLocalError: local variable 'text' referenced before assignment
